In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, IterableDataset
import torch.nn.functional as F
import chess
from tqdm import tqdm
import glob

torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
MODEL_NUMBER = 2
MODEL_VERSION = 1

In [3]:
class EvalNet(nn.Module):
    """
    Neural network model for evaluating chess positions.

    This model takes a chess position as input and predicts the evaluation score
    for that position. It consists of convolutional and fully connected layers.

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer.
        conv2 (nn.Conv2d): Second convolutional layer.
        fc1 (nn.Linear): First fully connected layer.
        fc2 (nn.Linear): Second fully connected layer.

    Methods:
        forward(x): Performs forward pass through the network.
    """

    def __init__(self) -> None:
        """
        Initializes the EvalNet class

        Args:
        - None

        Returns:
        - None
        """
        super(EvalNet, self).__init__()
        self.conv1 = nn.Conv2d(12, 16, kernel_size = 5, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(16, 24, kernel_size = 3, stride = 1, padding = 1) 
        self.fc1 = nn.Linear(24 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Performs a forward pass through the network.

        Args:
        - x (torch.Tensor): Input tensor of shape (batch_size, 12, 8, 8)
        
        Returns:
        - torch.Tensor: Output tensor of shape (batch_size, 1)
        """
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.fc1(x))
        return self.fc2(x)  

In [4]:
def fen_str_to_3d_tensor(fen: str) -> torch.Tensor:
    """
    Converts a FEN string representation of a chess position to a 3D tensor.

    Args:
    - fen (str): The FEN string representing the chess position.

    Returns:
    - torch.Tensor: A 3D tensor representing the chess position, where each element
                    corresponds to a piece on the board.

    Example:
        fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
        tensor = fen_str_to_3d_tensor(fen)
    """
    piece_to_int = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11,
    }

    board = np.zeros((12, 8, 8), dtype=np.float32)
    
    # Split the FEN string into parts ## 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
    fen_parts = fen.split(' ')
    fen_rows = fen_parts[0].split('/') # Only process the board position (the first part)
    
    for row_idx, row in enumerate(fen_rows):
        col_idx = 0
        for char in row:
            if char.isdigit():
                col_idx += int(char)
            else:
                piece = piece_to_int[char]
                board[piece, row_idx, col_idx] = 1
                col_idx += 1
    
    return torch.tensor(board)

In [5]:
path = "../Data/DataTrain"

letters_in = 'bcdefghijklmnopqrstuvwxyz'
letters_out = 'a'

csv_files_train = []
csv_files_val = []

for let in letters_in:
    csv_files_train.extend(glob.glob(f'{path}/Chess_Jan_{let}*'))
    csv_files_train.extend(glob.glob(f'{path}/Chess_Feb_{let}*'))
    csv_files_train.extend(glob.glob(f'{path}/Chess_Mar_{let}*')) # include in version 1-3
    # csv_files_train.extend(glob.glob(f'{path}/Chess_Apr_{let}*'))

for let_ in letters_out:
    csv_files_val.extend(glob.glob(f'{path}/Chess_Jan_{let_}*'))
    csv_files_val.extend(glob.glob(f'{path}/Chess_Feb_{let_}*'))
    csv_files_val.extend(glob.glob(f'{path}/Chess_Mar_{let_}*')) # include in version 1-3
    # csv_files_val.extend(glob.glob(f'{path}/Chess_Apr_{let_}*'))

In [6]:
def preprocess_inputs(DF: pl.DataFrame) -> torch.Tensor:
    """
    Proprocesses the input tensor into batches by applying the fen_str_to_3d_tensor function.
    """
    
    n_rows = DF.shape[0]

    inputs = torch.zeros(n_rows, 12, 8, 8)

    for i in range(n_rows):
        inputs[i] = fen_str_to_3d_tensor(DF['board'][i])
    
    return inputs

In [7]:
def Data(csv_file):
    
    df = pl.read_csv(csv_file, null_values=["None", "null", "#0", "#-0"])
    df = df.drop_nulls()

    df = df.with_columns(clip=pl.col("cp").clip(-10, 10))

    inputs = preprocess_inputs(df)
    targets = torch.tensor(df['cp'])

    return inputs, targets



class ChessIterableDataset(IterableDataset):
    def __init__(self, csv_files, chunksize=50000):
        """
        Initializes the ChessIterableDataset class.

        Args:
        - csv_files (list): List of CSV file paths.

        Returns:
        - None
        """
        self.csv_files = csv_files
        self.chunksize = chunksize

        
    def __len__(self):
        """
        Implements the length method.

        Returns:
        - int: Length of the dataset.
        """
        return sum(1 for _ in self.__iter__())


    def __iter__(self):
        """
        Implements the iterator logic.

        Returns:
        - Iterator object
        """
        return iter(self.csv_files)

In [8]:
def train(model, train_data_loader, val_data_loader, criterion, optimizer, num_epochs):
    """
    Trains model

    Parameters
    ----------
    model : torch.nn.Module
        model to be trained.
    train_data_loader : torch.utils.data.DataLoader
        training data.
    val_data_loader : torch.utils.data.DataLoader
        validation data.
    criterion : torch.nn.modules.loss._Loss
        loss function
    optimizer : torch.optim.Optimizer
        optimizer
    num_epochs : int
        Number of epochs

    Returns
    -------
    list
        average training loss for each epoch
    list
        average validation loss for each epoch

    """
    print(f'Begin Training! (on {device})')

    training_loss_history = []
    validation_loss_history = []

    try:
        for epoch in tqdm(range(num_epochs)):

            train_running_loss = 0.0
            val_running_loss = 0.0

            ## TRAINING PHASE =================================
            model.train()  # Set the model to training mode

            for i, csv_file in enumerate(csv_files_train):

                inputs, targets = Data(csv_file)

                inputs = inputs.to(device)
                targets = targets.to(device).unsqueeze(1)

                optimizer.zero_grad()
                train_outputs = model(inputs)
                train_batch_loss = criterion(train_outputs, targets)

                print(f"\t Training Batch Loss: {train_batch_loss}")

                train_batch_loss.backward()
                optimizer.step()

                train_running_loss += train_batch_loss.item()
            
            ## VALIDATION PHASE =================================
            model.eval()  # Set the model to evaluation mode
        
            with torch.no_grad():
                for i, csv_file in enumerate(csv_files_val):

                    inputs_val, targets_val = Data(csv_file)

                    inputs_val = inputs_val.to(device)
                    targets_val = targets_val.to(device).unsqueeze(1)
                    
                    val_outputs = model(inputs_val) # forward
                    val_batch_loss = criterion(val_outputs, targets_val)

                    print(f"\t Validation Batch Loss: {val_batch_loss}")

                    val_running_loss += val_batch_loss.item()

            print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_running_loss/len(csv_files_train):.5f}, Validation Loss: {val_running_loss/len(csv_files_val):.5f}')
            training_loss_history.append(train_running_loss/len(train_data_loader))
            validation_loss_history.append(val_running_loss/len(val_data_loader))
            
    except KeyboardInterrupt:
        print("Manual Stop: Finished Training Early!")
    finally:
        torch.save(model, f'models_autosave/autosave{MODEL_NUMBER}-{MODEL_VERSION}.pth')

    print(f'Finished Training!')

    return training_loss_history, validation_loss_history

In [9]:
# Create a dataset
dataset_train = ChessIterableDataset(csv_files_train)
dataset_val = ChessIterableDataset(csv_files_val)

# Create a data loader
train_data_loader = DataLoader(dataset_train, 
                               batch_size = 50000,
                            #    shuffle=True, # include in version 1-3
)


val_data_loader = DataLoader(dataset_val, 
                             batch_size = 50000,
                            #  shuffle=True, # include in version 1-3
)


model = EvalNet()
model = torch.load('models_autosave/autosave2-0.pth')
model = model.to(device)

criterion = nn.L1Loss() # nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.01)

train_loss_hist, valid_loss_hist = train(model, train_data_loader, val_data_loader, criterion, optimizer, num_epochs = 20)

Begin Training! (on cuda)


  0%|          | 0/20 [00:00<?, ?it/s]

	 Training Batch Loss: 2.701425552368164
	 Training Batch Loss: 2.844952344894409
	 Training Batch Loss: 2.478977918624878
	 Training Batch Loss: 2.7950637340545654
	 Training Batch Loss: 2.5609829425811768
	 Training Batch Loss: 2.7199442386627197
	 Training Batch Loss: 2.4520983695983887
	 Training Batch Loss: 2.622284412384033
	 Training Batch Loss: 2.526482582092285
	 Training Batch Loss: 2.6842849254608154
	 Training Batch Loss: 2.4676952362060547
	 Training Batch Loss: 2.532247304916382
	 Training Batch Loss: 2.674440622329712
	 Training Batch Loss: 2.5060136318206787
	 Training Batch Loss: 2.4860470294952393
	 Training Batch Loss: 2.550229549407959
	 Training Batch Loss: 2.5054118633270264
	 Training Batch Loss: 2.6299426555633545
	 Training Batch Loss: 2.5031967163085938
	 Training Batch Loss: 2.537966012954712
	 Training Batch Loss: 2.552461624145508
	 Training Batch Loss: 2.580718755722046
	 Training Batch Loss: 2.3462016582489014
	 Training Batch Loss: 2.404313802719116
	 Tr

In [ ]:
torch.save(model, f'models_5_3/model{MODEL_NUMBER}-{MODEL_VERSION}.pth')

In [ ]:
import pickle


with open('pickle/DL_2_3-3_train_loss_history.pkl', 'wb') as f:
    pickle.dump(train_loss_hist, f)

with open('pickle/DL_2_3-3_valid_loss_history.pkl', 'wb') as f:
    pickle.dump(valid_loss_hist, f)

In [ ]:
plt.figure()
plt.plot(train_loss_hist, label='Training Loss')
plt.plot(valid_loss_hist, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()